In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [2]:
from r_codes import geo_col,col1,col2,col3
from geo import stco_fips, metro_codes

In [3]:
col_head = {'o':geo_col,'1':col1,'2':col2,'3':col3}
col_join = ['LOGRECNO','STUSAB','FILEID','CHARITER']
col_data = ['STATE','COUNTY','GEOCODE','SUMLEV','POP100','HU100','P0010001','H0010001','NAME']
sumlev = [40,50,60,160,140]

In [4]:
# WILL NEED TO IDENTIFY STRING INDICATING STATE
folders = glob.glob('../data/red_20/*')

#use this to figure out where the statename is in the folder string
for folder in folders:
    print(folder[15:17])

### data pulling functions for regional data

In [ ]:
def make_state(state):
    files = glob.glob(f'../data/red_20/{state}2020.pl/*.pl')
    #for first file in folder
    df = pd.read_table(f'{files[0]}',sep='|',header=None,low_memory=False)
    df.columns = col_head[files[0][-8]]

    #for all other files
    for file in files[1:]:
        dff = pd.read_table(f'{file}',sep='|',header=None,low_memory=False)
        dff.columns = col_head[file[-8]] #CONFIRM WHERE FILE # WILL BE
        df = pd.merge(df,dff,left_on=col_join,right_on=col_join,how="inner")
    
    #reduce table size
    df = df[col_data] #just the data columns we need
    df = df[df.SUMLEV.isin(sumlev)].copy() #just the geo types we need
    df = df[df.GEOCODE.isin(metro_codes)].copy() #just the places in our region
    return df

In [ ]:
def make_table(folders):
    df = pd.DataFrame()
    for folder in folders:
        state = folder[15:17] #may need to change the location of where files are named
        dff = make_state(state)
        df = pd.concat([df,dff])
    return df

### make various tables

In [ ]:
#master regional table with counties, munis, and NYC tracts
master = make_table(folders)

In [ ]:
county = master[master['SUMLEV']==50].copy()
county.to_csv('output/counties_20.csv')

In [ ]:
#nyc tract to subborrough
nyc = master[master['SUMLEV']==140].copy()

In [6]:
nyc_recode = pd.read_csv('../data/geo/nyc_subbor_20.csv')
nyc_recode['GEOID20']=nyc_recode['GEOID20'].astype(str)

nyc = pd.merge(nyc_recode,nyc,left_on='GEOID20',right_on='GEOCODE',how='left')
nyc_sub = nyc[['STATEFP20','COUNTYFP20','Subbor_id','Subbor_nm','POP100','HU100','P0010001','H0010001']]

In [9]:
nyc_sub_test = nyc_sub.groupby(['STATEFP20','COUNTYFP20','Subbor_id','Subbor_nm']).sum().reset_index()

In [7]:
nyc_sub_test.head()

,STATEFP20,COUNTYFP20,TRACTCE20,GEOID20,NAME20,NAMELSAD20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,Subbor_nm,Subbor_id
0,36,81,152902,36081152902,1529.02,Census Tract,G5020,S,1032987,0,40.759314,-73.724016,QN Northeast,36081NE
1,36,81,152901,36081152901,1529.01,Census Tract,G5020,S,1399259,0,40.752330,-73.733654,QN Northeast,36081NE
2,36,81,150702,36081150702,1507.02,Census Tract,G5020,S,835426,0,40.766372,-73.730825,QN Northeast,36081NE
3,36,81,150701,36081150701,1507.01,Census Tract,G5020,S,877002,0,40.762201,-73.739702,QN Northeast,36081NE
4,36,81,148300,36081148300,1483.00,Census Tract,G5020,S,1334134,1047226,40.775859,-73.751970,QN Northeast,36081NE


In [ ]:
#rename columns to join with subplace table
nyc_sub_test = nyc_sub_test.rename(columns={'STATEFP20':'STATE','COUNTYFP20':'COUNTY',\
                                            'Subbor_id':'geoid','Subbor_nm':'name'})

In [ ]:
subpl20 = master[(master['SUMLEV']==160) or (master['SUMLEV']==60)]
subpl20.to_csv('output/subpl_20.csv')

In [11]:
#make adjusted subpl for calculation comparison over time
subpl_recode = pd.read_csv('../data/geo/subpl20.csv')

In [12]:
subpl_recode.head()

,stco,id_20,nm_20,geoid,name
0,9001,900104720,"Bethel town, Fairfield County, Connecticut",900104720,"Bethel town, Fairfield County, Connecticut"
1,9001,900108070,"Bridgeport town, Fairfield County, Connecticut",900108070,"Bridgeport town, Fairfield County, Connecticut"
2,9001,900108980,"Brookfield town, Fairfield County, Connecticut",900108980,"Brookfield town, Fairfield County, Connecticut"
3,9001,900118500,"Danbury town, Fairfield County, Connecticut",900118500,"Danbury town, Fairfield County, Connecticut"
4,9001,900118850,"Darien town, Fairfield County, Connecticut",900118850,"Darien town, Fairfield County, Connecticut"


In [15]:
subpl_recode.dtypes

stco      int64
id_20     int64
nm_20    object
geoid     int64
name     object
dtype: object

In [ ]:
subpl20['joinid'] = subpl20.GEOCODE.astype(int)
subpl_adj = pd.merge(subpl_recode,subpl20,left_on='id_20',right_on='joinid',how='left')

In [ ]:
subpl_adj = subpl_adj[['STATE','COUNTY','geoid','name','POP100','HU100','P0010001','H0010001']]
subpl_adj_2 = subpl_adj.groupby(['STATE','COUNTY','geoid','name']).sum().reset_index()

In [ ]:
subpl_final = pd.concat([nyc_sub_test,subpl])